In [1]:
import pandas as pd
import rasterio as rio
import networkit as nk
import networkx as nx

import sys
sys.path.append('C:/Users/pj276/Projects/undp_connectivity/code/functions')
import reskernel_functions as rkf
import numpy as np, time
from numpy import copy
import rasterio as rio
import networkit as nk
from itertools import zip_longest
from itertools import chain

In [2]:
# Functions
# Coords is a two column array where 1st column is x's and 2nd column is y's
def cell_indices_from_coords_nowindow(src, src_data, coords):
    # proceed if there are any target values in the array
    if coords.ndim > 0: #coords.shape[0] > 0:
        # get rows and columns of coordinates
        row, col = rio.transform.rowcol(src.transform, coords[:,0], coords[:,1])
        cinds = np.array((row,col)).transpose()
    else:
        cinds = np.array(np.nan)
    return cinds

In [3]:
# Coordinate reference system
# crsString = 'ESRI:102028'
# Cell size
cSize = 500

# xy coordinate file
xyf = 'C:/Users/pj276/Projects/CDPOP/data/sabah_example1_1686679883/batchrun0mcrun0/XY200.csv'
xy = pd.read_csv(xyf)

# resistance grid file
rg = 'C:/Users/pj276/Projects/UNICOR/unicor/resist_sabah_example_pro.tif'
with rio.open(rg) as src:
    r = src.read(1)
    # Convert to int16
    r = copy(r).astype(np.int16)

In [4]:
print("Converting image to graph", flush=True)
tic = time.perf_counter()
nkG, nodeids, idmap = rkf.image_to_graph(r, cSize, -9999)
toc = time.perf_counter()
print(f"Converting image to graph took {toc - tic:0.4f} seconds", flush=True)
print("number of edges = ", str(nkG.numberOfEdges()), flush=True)

Converting image to graph
Converting image to graph took 13.3193 seconds
number of edges =  1148287


In [5]:
with rio.open(rg) as src:
    # Get source indices as rows and columns
    cinds = cell_indices_from_coords_nowindow(src, r, np.array(xy))
    # Convert to 1D
    sources = [(i[0]*r.shape[1])+i[1] for i in cinds]
    # Convert to networkit nodeids
    sources = [idmap[n] for n in sources]
    print("number of sources = " + str(len(sources)), flush=True)
print(sources[0])
print(sources[2])

number of sources = 244
1234
2596


In [24]:
nk.setNumberOfThreads(3)

In [ ]:
sPaths = []
source = 2596
tic = time.perf_counter()
dij = nk.distance.Dijkstra(nkG, source, storePaths=True, storeNodesSortedByDistance=False)
dij.run()
for target in sources:
    # The distance from source to target node
    #print(dij.distance(target))
    # The number of paths
    #print(dij.numberOfPaths(target))
    # Returns shortest paths from source to target
    sPs = dij.getPaths(target)
    sPaths.append(sPs)
toc = time.perf_counter()
print(f"Calculating shortest paths took {toc - tic:0.4f} seconds", flush=True)

In [60]:
print(len(sPaths))
merged = list(chain.from_iterable(sPaths))
merged = list(chain.from_iterable(merged))
print(merged)

244
[1408, 1409, 1410, 1411, 1412, 1413, 1414, 1415, 1416, 1417, 1360, 1297, 1298, 1233, 1234, 1408, 1409, 1410, 1411, 1412, 1413, 1414, 1415, 1416, 1417, 1360, 1297, 1298, 1299, 1234, 1408, 1409, 1410, 1411, 1412, 1413, 1414, 1415, 1416, 1417, 1360, 1361, 1298, 1233, 1234, 1408, 1409, 1410, 1411, 1412, 1413, 1414, 1415, 1416, 1417, 1360, 1361, 1298, 1299, 1234, 1408, 1409, 1410, 1411, 1412, 1413, 1414, 1415, 1416, 1417, 1418, 1361, 1298, 1233, 1234, 1408, 1409, 1410, 1411, 1412, 1413, 1414, 1415, 1416, 1417, 1418, 1361, 1298, 1299, 1234, 1408, 1407, 1406, 1405, 1404, 1403, 1402, 1401, 1400, 1399, 1398, 1397, 1396, 1395, 1394, 1393, 1392, 1391, 1390, 1389, 1388, 1387, 1386, 1446, 1445, 1444]


In [52]:
print(len(merged))
print((0.0071*244))

91
1.7324000000000002


In [7]:
tic = time.perf_counter()
aa = nk.distance.SPSP(nkG, sources)
aa.run()
bb = aa.getDistances()
toc = time.perf_counter()
print(f"Calculating shortest paths took {toc - tic:0.4f} seconds", flush=True)

Calculating shortest paths took 7.3161 seconds


In [26]:
# Get distances to other source nodes only
ccList = []
for i in bb:
    cc = [ i[s] for s in sources]
    ccList.append(cc)
# NEXT
# Idenitify node pairs that are less than some threshold distance
# Use these node pairs in Dijkstra calculations

In [ ]:
dij = nk.distance.Dijkstra(nkG, 2596, storePaths=True, storeNodesSortedByDistance=False)
dij.run()
dij.getPaths(289361)

In [25]:
nk.getCurrentNumberOfThreads()

1

In [22]:
atest = [ bb[100][i] for i in sources]
print(atest)

[1.7976931348623157e+308, 1.7976931348623157e+308, 1661398.0959472656, 1646708.7630004883, 1375242.5042114258, 1377918.1693725586, 1308577.7693481445, 1127599.848083496, 1057991.4631958008, 1107334.0859375, 966736.6298217773, 1022350.5893554688, 878443.2131958008, 945836.6115722656, 983190.1647949219, 1107930.801208496, 1176642.21875, 1024046.0241088867, 1155569.8876953125, 1006993.9865722656, 779090.3999633789, 1219040.9724731445, 899500.8251953125, 848213.5055541992, 968753.1340332031, 989981.0559082031, 942860.4735717773, 843599.5454711914, 904355.6414794922, 801726.9617919922, 792271.1165771484, 879809.700378418, 841935.3319702148, 922129.5047607422, 810628.9831542969, 757655.3724975586, 899962.2431030273, 933796.7662353516, 956520.3768310547, 834108.6896362305, 778790.1481933594, 1026269.1141357422, 939956.665222168, 1094638.4317626953, 946304.1258544922, 1036847.5423583984, 808024.9067382812, 954746.5135498047, 969435.1126708984, 967857.426574707, 984928.4943237305, 1301607.20861

In [32]:
print(ccList[100])



[1.7976931348623157e+308, 1.7976931348623157e+308, 1661398.0959472656, 1646708.7630004883, 1375242.5042114258, 1377918.1693725586, 1308577.7693481445, 1127599.848083496, 1057991.4631958008, 1107334.0859375, 966736.6298217773, 1022350.5893554688, 878443.2131958008, 945836.6115722656, 983190.1647949219, 1107930.801208496, 1176642.21875, 1024046.0241088867, 1155569.8876953125, 1006993.9865722656, 779090.3999633789, 1219040.9724731445, 899500.8251953125, 848213.5055541992, 968753.1340332031, 989981.0559082031, 942860.4735717773, 843599.5454711914, 904355.6414794922, 801726.9617919922, 792271.1165771484, 879809.700378418, 841935.3319702148, 922129.5047607422, 810628.9831542969, 757655.3724975586, 899962.2431030273, 933796.7662353516, 956520.3768310547, 834108.6896362305, 778790.1481933594, 1026269.1141357422, 939956.665222168, 1094638.4317626953, 946304.1258544922, 1036847.5423583984, 808024.9067382812, 954746.5135498047, 969435.1126708984, 967857.426574707, 984928.4943237305, 1301607.20861

In [14]:
749*923

691327